### CaboChaの出力結果をsqlite3に保存する
分解結果をsqlite3に保存する。

### データベース接続の関数定義

In [42]:
import json
import sqlite3

conn = None

# データベース接続
def connect():
    # global変数でconnを呼び出し
    global conn
    # データベースの場所を指定
    conn = sqlite3.connect('../../NaturalLanguageProcessing/data/sqlite3/sqlite3')

# データベース接続終了
def close():
#   終了
    conn.close()

# テーブル作成
def create_table():
#   executeでSQL構文作成、docsがあれば削除
    conn.execute('DROP TABLE IF EXISTS docs')
#   docsテーブルを新規作成
    conn.execute('''CREATE TABLE docs (
            id          INTEGER PRIMARY KEY AUTOINCREMENT,
            content     TEXT,
            meta_info   BLOB,
            sentence    BLOB,
            chunk       BLOB,
            token       BLOB
        )''')

# データをインサートする
def db_load(values):
    
#   valuesに指定したパラメータ順序またはマッピングを?に入れて実行する
    conn.executemany(
        'INSERT INTO docs (content, meta_info) VALUES (?,?)',
        values)
#   確定
    conn.commit()

# 一部のデータを見る
def get(doc_id, fl):
#   .fetchone()でカーソルの次の行を取得
    row_ls = conn.execute(f"SELECT {','.join(fl)} FROM docs WHERE id = {doc_id}").fetchone()
    row_dict = {}
#   flとrow_lsで抜き出したデータをzipする
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

# id番号を抜き出す
def get_all_ids(limit, offset=0):
#   limitで取得上限、OFFSETで開始位置を指定してデータを抜き出す。そのデータの1番目id番号を抜き出す
    return [record[0] for record in conn.execute('SELECT id FROM docs LIMIT ? OFFSET ?', (limit, offset))]

# アノテーションを設定
def set_annotation(doc_id, name, value):
#   docsのid行をwhere idで指定し、その行にname = valueのアノテーションを追加　
    conn.execute('UPDATE docs SET {0} = ? where id = ?'.format(name), (json.dumps(value), doc_id))
#   確定
    conn.commit()

# アノテーションを確認
def get_annotation(doc_id, name):
#   docsのid行をwhere idで指定しnameから取り出す
    row = conn.execute('SELECT {0} FROM docs WHERE id = ?'.format(name), (doc_id,)).fetchone()
    if row[0] is not None:
        return json.loads(row[0])
    else:
        return []

### Cabocha処理の関数を定義


In [23]:
# 正規表現操作を行うライブラリ
import re
# CaboChaをインポート
import CaboCha

# CaboChaをインスタンス化
cabocha = CaboCha.Parser('-n1')
# 。!<_EOS_>を文の文末としている為正規表現にまとめ文の区切り時に使用する。
ptn_sentence = re.compile(r'(^|。|！|<__EOS__>)\s*(.+?)(?=(。|！|<__EOS__>))', re.M)

# 正規表現を使用して文末で分割する関数定義
def split_into_sentences(text):
    sentences = []
    for m in ptn_sentence.finditer(text):
        sentences.append((m.group(2), m.start(2)))
    return sentences

# 係り受け木を構築し、ディキショナリで返す関数定義
def parse_sentence(sentence_str, sentence_begin, chunks, tokens):
    tree = cabocha.parse(sentence_str)

    offset = sentence_begin
    chunk_id_offset = len(chunks)
    text = sentence_str
    for i in range(tree.chunk_size()):
        chunk = tree.chunk(i)
        chunk_begin = None
        for j in range(
                chunk.token_pos,
                chunk.token_pos + chunk.token_size):
            token = tree.token(j)
            features = token.feature.split(',')
            token_begin = text.find(token.surface) + offset
            token_end = token_begin + len(token.surface)
            if chunk_begin is None:
                chunk_begin = token_begin

            tokens.append({
                'begin': token_begin,
                'end':   token_end,
                'lemma': features[-3],
                'POS':   features[0],
                'POS2':  features[1],
                'NE':    token.ne,
            })

            text = text[token_end-offset:]
            offset = token_end

        chunk_end = token_end
        if chunk.link == -1:
            link = -1
        else:
            # チャンクのIDを手前に出現したチャンス数だけずらす
            link = chunk.link + chunk_id_offset
        chunks.append({
            'begin':    chunk_begin,
            'end':      chunk_end,
            'link':     ('chunk', link),
        })

# 上記２つの関数を利用し文の情報を生成する関数定義
def parse(text):
    sentences = []
    chunks = []
    tokens = []
    sentence_begin = 0

    # 正規表現を使用して文末で分割
    for sentence_str, sentence_begin in split_into_sentences(text):
        # 係り受け木を構築し、ディキショナリで返す
        parse_sentence(sentence_str, sentence_begin, chunks, tokens)
        sentence_end = chunks[-1]['end']

        sentences.append({
            'begin':    sentence_begin,
            'end':      sentence_end,
        })

    return sentences, chunks, tokens


### Sqlite3にデータを保存する。

### docsテーブル確認
 * ディレクトリ：../../NaturalLanguageProcessing/data/sqlite3/sqlite3
 * テーブル名：docs

| #  | 項目                                      | 型                                   | 項目説明                                 |
| -- | ----------------------------------------- | ------------------------------------ | -----------------------------------------|
| 1  | id                                        | INTEGER                              | 既に保存                                 |
| 2  | content                                   | TEXT                                 | 既に保存                                 |
| 3  | meta_info                                 | BLOB                                 | 既に保存                                 |
| 4  | <font color="red"><b>sentence</b></font>  | <font color="red"><b>BLOB</b></font> | <font color="red"><b>今回追加</b></font> |
| 5  | <font color="red"><b>chunk</b></font>     | <font color="red"><b>BLOB</b></font> | <font color="red"><b>今回追加</b></font> |
| 6  | <font color="red"><b>token</b></font>     | <font color="red"><b>BLOB</b></font> | <font color="red"><b>今回追加</b></font> |

In [38]:
connect()
for doc_id in get_all_ids(limit=-1):
    row = get(doc_id, fl=['content'])
    text = row['content']
    sentences, chunks, tokens = parse(text)
    print('parsed: doc_id =', doc_id)

    set_annotation(doc_id, 'sentence', sentences)
    set_annotation(doc_id, 'chunk',    chunks)
    set_annotation(doc_id, 'token',    tokens)

close()


parsed: doc_id = 1
parsed: doc_id = 2
parsed: doc_id = 3
parsed: doc_id = 4
parsed: doc_id = 5
parsed: doc_id = 6
parsed: doc_id = 7
parsed: doc_id = 8
parsed: doc_id = 9
parsed: doc_id = 10
parsed: doc_id = 11
parsed: doc_id = 12
parsed: doc_id = 13
parsed: doc_id = 14
parsed: doc_id = 15
parsed: doc_id = 16
parsed: doc_id = 17
parsed: doc_id = 18
parsed: doc_id = 19
parsed: doc_id = 20
parsed: doc_id = 21
parsed: doc_id = 22
parsed: doc_id = 23
parsed: doc_id = 24
parsed: doc_id = 25
parsed: doc_id = 26
parsed: doc_id = 27
parsed: doc_id = 28
parsed: doc_id = 29
parsed: doc_id = 30
parsed: doc_id = 31
parsed: doc_id = 32
parsed: doc_id = 33
parsed: doc_id = 34
parsed: doc_id = 35
parsed: doc_id = 36
parsed: doc_id = 37
parsed: doc_id = 38
parsed: doc_id = 39
parsed: doc_id = 40
parsed: doc_id = 41
parsed: doc_id = 42
parsed: doc_id = 43
parsed: doc_id = 44
parsed: doc_id = 45
parsed: doc_id = 46
parsed: doc_id = 47
parsed: doc_id = 48
parsed: doc_id = 49
parsed: doc_id = 50
parsed: d